<a href="https://colab.research.google.com/github/KushalSethia30/MarketingResearchMethods-Kushal-Sethia-Jio-Institute/blob/main/Accuracy_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from ucimlrepo import fetch_ucirepo
import numpy as np

# Fetch dataset
phishing_websites = fetch_ucirepo(id=327)
X = phishing_websites.data.features
y = phishing_websites.data.targets

# Reshape y to 1D array
y = y.values.ravel()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data (important for some algorithms like Logistic Regression, SVM, ANN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'SVM': SVC(),
    'Artificial Neural Network': MLPClassifier(max_iter=500),
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Fisher Discriminant': LDA()
}

# For storing results
feature_importance = {}
metrics_table = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Train and evaluate models
metrics_list = []  # List to collect metrics for concatenation

for name, model in models.items():
    # Use scaled data for models that require it
    if name in ['Logistic Regression', 'K-Nearest Neighbors', 'SVM', 'Artificial Neural Network', 'Fisher Discriminant']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Collect metrics in a list
    metrics_list.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })

    # Feature importance for relevant models
    if hasattr(model, 'coef_'):  # For Logistic Regression and LDA
        importances = np.abs(model.coef_[0])
        indices = np.argsort(importances)[-10:][::-1]
        feature_importance[name] = [(X.columns[i], importances[i]) for i in indices]
    elif hasattr(model, 'feature_importances_'):  # For tree-based models
        importances = model.feature_importances_
        indices = np.argsort(importances)[-10:][::-1]
        feature_importance[name] = [(X.columns[i], importances[i]) for i in indices]

# Convert the list of metrics into a DataFrame
metrics_table = pd.DataFrame(metrics_list)

# Output the metrics table
print("\nAccuracy Metrics:")
print(metrics_table)

# Generate feature importance table
print("\nFeature Importance (Top 10) for Each Model:")
for model, features in feature_importance.items():
    print(f"\n{model}:")
    for feature, importance in features:
        print(f"Feature: {feature}, Importance: {importance:.4f}")


Accuracy Metrics:
                       Model  Accuracy  Precision    Recall  F1-Score
0        Logistic Regression  0.922219   0.931253  0.932239  0.931746
1        K-Nearest Neighbors  0.932469   0.940243  0.941239  0.940741
2                        SVM  0.946639   0.946764  0.960296  0.953482
3  Artificial Neural Network  0.969249   0.968537  0.977766  0.973130
4              Random Forest  0.968345   0.963136  0.982001  0.972477
5              Decision Tree  0.956286   0.957503  0.966120  0.961792
6                   AdaBoost  0.939403   0.942812  0.951297  0.947036
7        Fisher Discriminant  0.918601   0.917483  0.941768  0.929467

Feature Importance (Top 10) for Each Model:

Logistic Regression:
Feature: url_of_anchor, Importance: 2.2940
Feature: prefix_suffix, Importance: 2.1905
Feature: sslfinal_state, Importance: 1.4951
Feature: web_traffic, Importance: 0.6651
Feature: sfh, Importance: 0.6639
Feature: links_in_tags, Importance: 0.6444
Feature: having_ip_address, Importanc